In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**implementation by https://www.geeksforgeeks.org/**text-generation-using-recurrent-long-short-term-memory-network/ below

In [3]:
from __future__ import absolute_import, division,print_function, unicode_literals
 
import numpy as np
import tensorflow as tf
 
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
 
from tensorflow.keras.optimizers import RMSprop
 
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import random
import sys, spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [26]:
text = open('/content/drive/MyDrive/Data science/job_tests/Ai bees/comments_generator_data/news_post.txt').read()

In [7]:
import string

punct = string.punctuation
stopwords =list(STOP_WORDS)
def data_cleaning(sentence):
    doc = nlp(sentence)
    tokens =[]
    for token in doc:
        if token .lemma_ !="-PRON-":
            temp= token.lemma_.lower().strip()
        else: 
            temp= token.lower_
        tokens.append(temp)
    clean_tokens = []
    for token in tokens:
        if token not in stopwords and token not in  punct:
            clean_tokens.append(token)
    return clean_tokens

In [8]:
import string
def clean_data(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower for word in tokens]
  return tokens

https://machinelearningmastery.com/clean-text-machine-learning-python/

In [13]:
# split into words by white space
words = text.split()
print(words[:100])

['Ad', 'sales', 'boost', 'Time', 'Warner', 'profit', 'Quarterly', 'profits', 'at', 'US', 'media', 'giant', 'TimeWarner', 'jumped', '76%', 'to', '$1.13bn', '(Â£600m)', 'for', 'the', 'three', 'months', 'to', 'December,', 'from', '$639m', 'year-earlier.', 'The', 'firm,', 'which', 'is', 'now', 'one', 'of', 'the', 'biggest', 'investors', 'in', 'Google,', 'benefited', 'from', 'sales', 'of', 'high-speed', 'internet', 'connections', 'and', 'higher', 'advert', 'sales.', 'TimeWarner', 'said', 'fourth', 'quarter', 'sales', 'rose', '2%', 'to', '$11.1bn', 'from', '$10.9bn.', 'Its', 'profits', 'were', 'buoyed', 'by', 'one-off', 'gains', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'Warner', 'Bros,', 'and', 'less', 'users', 'for', 'AOL.', 'Time', 'Warner', 'said', 'on', 'Friday', 'that', 'it', 'now', 'owns', '8%', 'of', 'search-engine', 'Google.', 'But', 'its', 'own', 'internet', 'business,', 'AOL,']


In [14]:
# split based on words only
import re
words = re.split(r'\W+', text)
print(words[:100])

['Ad', 'sales', 'boost', 'Time', 'Warner', 'profit', 'Quarterly', 'profits', 'at', 'US', 'media', 'giant', 'TimeWarner', 'jumped', '76', 'to', '1', '13bn', 'Â', '600m', 'for', 'the', 'three', 'months', 'to', 'December', 'from', '639m', 'year', 'earlier', 'The', 'firm', 'which', 'is', 'now', 'one', 'of', 'the', 'biggest', 'investors', 'in', 'Google', 'benefited', 'from', 'sales', 'of', 'high', 'speed', 'internet', 'connections', 'and', 'higher', 'advert', 'sales', 'TimeWarner', 'said', 'fourth', 'quarter', 'sales', 'rose', '2', 'to', '11', '1bn', 'from', '10', '9bn', 'Its', 'profits', 'were', 'buoyed', 'by', 'one', 'off', 'gains', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'Warner', 'Bros', 'and', 'less', 'users', 'for', 'AOL', 'Time', 'Warner', 'said', 'on', 'Friday', 'that', 'it', 'now', 'owns', '8', 'of', 'search']


In [15]:
# split into words by white space
words = text.split()
# remove punctuation from each word
import string
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in words]
print(stripped[:100])

['Ad', 'sales', 'boost', 'Time', 'Warner', 'profit', 'Quarterly', 'profits', 'at', 'US', 'media', 'giant', 'TimeWarner', 'jumped', '76', 'to', '113bn', 'Â£600m', 'for', 'the', 'three', 'months', 'to', 'December', 'from', '639m', 'yearearlier', 'The', 'firm', 'which', 'is', 'now', 'one', 'of', 'the', 'biggest', 'investors', 'in', 'Google', 'benefited', 'from', 'sales', 'of', 'highspeed', 'internet', 'connections', 'and', 'higher', 'advert', 'sales', 'TimeWarner', 'said', 'fourth', 'quarter', 'sales', 'rose', '2', 'to', '111bn', 'from', '109bn', 'Its', 'profits', 'were', 'buoyed', 'by', 'oneoff', 'gains', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'Warner', 'Bros', 'and', 'less', 'users', 'for', 'AOL', 'Time', 'Warner', 'said', 'on', 'Friday', 'that', 'it', 'now', 'owns', '8', 'of', 'searchengine', 'Google', 'But', 'its', 'own', 'internet', 'business', 'AOL']


In [16]:
# convert to lower case
words = [word.lower() for word in words]
print(words[:100])

['ad', 'sales', 'boost', 'time', 'warner', 'profit', 'quarterly', 'profits', 'at', 'us', 'media', 'giant', 'timewarner', 'jumped', '76%', 'to', '$1.13bn', '(â£600m)', 'for', 'the', 'three', 'months', 'to', 'december,', 'from', '$639m', 'year-earlier.', 'the', 'firm,', 'which', 'is', 'now', 'one', 'of', 'the', 'biggest', 'investors', 'in', 'google,', 'benefited', 'from', 'sales', 'of', 'high-speed', 'internet', 'connections', 'and', 'higher', 'advert', 'sales.', 'timewarner', 'said', 'fourth', 'quarter', 'sales', 'rose', '2%', 'to', '$11.1bn', 'from', '$10.9bn.', 'its', 'profits', 'were', 'buoyed', 'by', 'one-off', 'gains', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'warner', 'bros,', 'and', 'less', 'users', 'for', 'aol.', 'time', 'warner', 'said', 'on', 'friday', 'that', 'it', 'now', 'owns', '8%', 'of', 'search-engine', 'google.', 'but', 'its', 'own', 'internet', 'business,', 'aol,']


In [18]:
# split into words
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text)
print(tokens[:100])

LookupError: ignored

https://www.geeksforgeeks.org/python-efficient-text-data-cleaning/

In [40]:
#Escaping out HTML characters
from html.parser import HTMLParser
import html
#text="I enjoyd the event which took place yesteday & I lovdddd itttt ! The link to the show is http://t.co/4ftYom0i It's awesome you'll luv it #HadFun #Enjoyed BFN GN" 
text=html.unescape(text)
print("After removing HTML characters ",text[:150])

After removing HTML characters  Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, fro


In [42]:
#library for regular expressions
import re

# remove hyperlinks
text = re.sub(r'https?:\/\/.\S+', "", text)
# remove hashtags
# only removing the hash # sign from the word
text = re.sub(r'#', '', text)

# remove old style retext text "RT"
text = re.sub(r'^RT[\s]+', '', text)

print("After removing Hashtags,URLs and Styles the text ",text[:150])


After removing Hashtags,URLs and Styles the text  Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, fro


Standardizing and Spell Check: There might be spelling errors in the text or it might not be in the correct format. For example – “drivng” for “driving” or “I misssss this” for “I miss this”.

In [22]:
 pip install autocorrect

     |████████████████████████████████| 622 kB 27.5 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.5.0-py3-none-any.whl size=621851 sha256=c2db984fb0c2ec56a7d814d76c75eb67fa3adcd2f76dee5cce0b2122f4d03005
  Stored in directory: /root/.cache/pip/wheels/3d/8e/bd/f6fd900a056a031bf710a00bca338d86f43b83f0c25ab5242f
Successfully built autocorrect


In [43]:
import itertools
#One letter in a word should not be present more than twice in continuation
text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
print("After standardizing the text is ",text[:100])

from autocorrect import Speller
spell = Speller(lang='en')
#spell check
text=spell(text)
print("After Spell check the text is ",text[:150])


After standardizing the text is  Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.1
After Spell check the text is  Ad sales boost Time Warner profit

Quarterly profits at Up media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, fro


# Remove Stopwords 
Stop words are the words which occur frequently in the text but add no significant meaning to it. For this, we will be using the nltk library which consists of modules for pre-processing data. It provides us with a list of stop words. You can create your own stopwords list as well according to the use case.

In [50]:
import nltk
#download the stopwords from nltk using
nltk.download('stopwords')
#import stopwords
from nltk.corpus import stopwords

#import english stopwords list from nltk
stopwords_eng = stopwords.words('english')

text_tokens=text.split()
text_list=[]
#remove stopwords
for word in text_tokens:
	if word not in stopwords_eng:
		text_list.append(word)

print("text_list ",text[:150])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
text_list  Ad sales boost Time Warner profit

Quarterly profits at Up media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, fro


In [53]:
import string         
clean_text=[]
#remove punctuations
for word in text_list:
    if word not in string.punctuation:
        clean_text.append(word)
 
print("clean_text = ",text[:500])

clean_text =  Ad sales boost Time Warner profit

Quarterly profits at Up media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buried by one-off gains which offset a profit dip at Warner Bros, and less users for OL.

Time W


In [54]:
# Storing all the unique characters present in the text
#text= clean_data(data)
vocabulary = sorted(list(set(text)))

# Creating dictionaries to map each character to an index
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary))

print(vocabulary)


['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', '¿', 'Â', 'ã', 'é', 'ó', 'ô']


In [55]:
# Dividing the text into subsequences of length max_length
# So that at each time step the next max_length characters
# are fed into the network
max_length = 100
steps = 5
sentences = []
next_chars = []
for i in range(0, len(text) - max_length, steps):
	sentences.append(text[i: i + max_length])
	next_chars.append(text[i + max_length])
	
# Hot encoding each character into a boolean vector
X = np.zeros((len(sentences), max_length, len(vocabulary)), dtype = np.bool)
y = np.zeros((len(sentences), len(vocabulary)), dtype = np.bool)
for i, sentence in enumerate(sentences):
	for t, char in enumerate(sentence):
		X[i, t, char_to_indices[char]] = 1
	y[i, char_to_indices[next_chars[i]]] = 1


In [56]:
# Building the LSTM network for the task
model = Sequential()
model.add(LSTM(128, input_shape =(max_length, len(vocabulary))))
model.add(Dense(len(vocabulary)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr = 0.01)
model.compile(loss ='categorical_crossentropy', optimizer = optimizer,metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [57]:
# Helper function to sample an index from a probability array
def sample_index(preds, temperature = 1.0):
	preds = np.asarray(preds).astype('float64')
	preds = np.log(preds) / temperature
	exp_preds = np.exp(preds)
	preds = exp_preds / np.sum(exp_preds)
	probas = np.random.multinomial(1, preds, 1)
	return np.argmax(probas)


In [67]:
# Helper function to generate text after the end of each epoch
def on_epoch_end(epoch, logs):
	print()
	print('----- Generating text after Epoch: % d' % epoch)

	start_index = random.randint(0, len(text) - max_length - 1)
	for diversity in [0.2, 0.5, 1.0, 1.2]:
		print('----- diversity:', diversity)

		generated = ''
		sentence = text[start_index: start_index + max_length]
		generated += sentence
		print('----- Generating with seed: "' + sentence + '"')
		sys.stdout.write(generated)

		for i in range(150):
			x_pred = np.zeros((1, max_length, len(vocabulary)))
			for t, char in enumerate(sentence):
				x_pred[0, t, char_to_indices[char]] = 1.

			preds = model.predict(x_pred, verbose = 0)[0]
			next_index = sample_index(preds, diversity)
			next_char = indices_to_char[next_index]

			generated += next_char
			sentence = sentence[1:] + next_char

			sys.stdout.write(next_char)
			sys.stdout.flush()
		print()
print_callback = LambdaCallback(on_epoch_end = on_epoch_end)


In [59]:
# Defining a helper function to save the model after each epoch
# in which the loss decreases
filepath = "/content/drive/MyDrive/Data science/job_tests/Ai bees/comments_generator_data/weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor ='loss',
							verbose = 1, save_best_only = True,
							mode ='min')


In [60]:
# Defining a helper function to reduce the learning rate each time
# the learning plateaus
reduce_alpha = ReduceLROnPlateau(monitor ='loss', factor = 0.2,
							patience = 1, min_lr = 0.001)
callbacks = [print_callback, checkpoint, reduce_alpha]


In [66]:
# Training the LSTM model
model.fit(X, y, batch_size = 512, epochs = 1, callbacks = callbacks)


1963/1963 [==============================] - 1833s 933ms/step - loss: 1.7932 - accuracy: 0.4799

----- Generating text after Epoch:  0
----- diversity: 0.2
----- Generating with seed: "put Apple's Newton and the first Palm Pilot there as the front runners for portable computing, and p"
put Apple's Newton and the first Palm Pilot there as the front runners for portable computing, and production and the content to the content to the police to the more the securice to be the search of the position and the content to the the production and the production and the production of the content to be a seen to the content to more the securice of the production and the content to the securice of the content and the content to the content to the presented to the content to the production t
----- diversity: 0.5
----- Generating with seed: "put Apple's Newton and the first Palm Pilot there as the front runners for portable computing, and p"
put Apple's Newton and the first Palm Pilot there as the fro

In [69]:
 # Defining a utility function to generate new and random text based on the
# network's learnings
def generate_text(length, diversity):
	# Get random starting text
	start_index = random.randint(0, len(text) - max_length - 1)
	generated = ''
	sentence = text[start_index: start_index + max_length]
	generated += sentence
	for i in range(length):
			x_pred = np.zeros((1, max_length, len(vocabulary)))
			for t, char in enumerate(sentence):
				x_pred[0, t, char_to_indices[char]] = 1.

			preds = model.predict(x_pred, verbose = 0)[0]
			next_index = sample_index(preds, diversity)
			next_char = indices_to_char[next_index]

			generated += next_char
			sentence = sentence[1:] + next_char
	return generated

print(generate_text(15, 0.2))


biggest gas company in the world, took over nuclear fuel giant Atomstroieksport in October 2004. The content the ti
